# Scraping using Geocoding API of OpenStreet Maps(OSM)<br>
We will find address using web scrapping and using python

In [ ]:
# import nominatim api
from geopy.geocoders import Nominatim

In [ ]:
# activate nominatim geocoder
locator = Nominatim(user_agent = "http")
# type any address
location = locator.geocode("Sree Padmanabhaswamy Temple")

In [ ]:
# get the latitide and longitude of the place
print(f'Address : {location.address}')
print(f'Latitude : {location.latitude}\nLongitude : {location.longitude}')

Address : Sree Padmanabhaswamy Temple, Padmavilasam Road, Padmanagar, Fort, Thiruvananthapuram, Kerala, 695001, India
Latitude : 8.482769950000002
Longitude : 76.94370162822179


In [ ]:
# print all the information about ebtered address thtat the API has to offer
location.raw

{'boundingbox': ['8.482634', '8.4828919', '76.9434819', '76.9439228'],
 'class': 'amenity',
 'display_name': 'Sree Padmanabhaswamy Temple, Padmavilasam Road, Padmanagar, Fort, Thiruvananthapuram, Kerala, 695001, India',
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//place_of_worship_unknown3.p.20.png',
 'importance': 0.30100000000000005,
 'lat': '8.482769950000002',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'lon': '76.94370162822179',
 'osm_id': 480106233,
 'osm_type': 'way',
 'place_id': 207630336,
 'type': 'place_of_worship'}

In [ ]:
# location 2
my_college = locator.geocode("COllege Of Engineering Pune")
# get the latitide and longitude of the place
print(f'Address : {my_college.address}')
print(f'Latitude : {my_college.latitude}\nLongitude : {my_college.longitude}')

Address : College of Engineering, Pune, Old Pune–Mumbai Highway, Shivajinagar, Pune City, Pune District, Maharashtra, 411001, India
Latitude : 18.52930005
Longitude : 73.85682537025511


In [ ]:
my_college.raw

{'boundingbox': ['18.5285846', '18.5315557', '73.8545636', '73.8586755'],
 'class': 'amenity',
 'display_name': 'College of Engineering, Pune, Old Pune–Mumbai Highway, Shivajinagar, Pune City, Pune District, Maharashtra, 411001, India',
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//education_school.p.20.png',
 'importance': 0.7813885782638657,
 'lat': '18.52930005',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'lon': '73.85682537025511',
 'osm_id': 3718263,
 'osm_type': 'relation',
 'place_id': 282449018,
 'type': 'college'}

# Get the data - BBC Weather location service<br>
We will scrape the location ID of any place that we enter.

In [ ]:
import os

import requests # get request from webpage
import json # convert API to json format

from urllib.parse import urlencode
import numpy as np
import pandas as pd
import re
from datetime import datetime

from bs4 import BeautifulSoup as bs

In [ ]:
# Enter city name and convert url into compact form
city_name = "Pune"
location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
    'stack': 'aws',
    'locale': 'en',
    'filter': 'international',
    'place-types': 'settlement,airport,district',
    'order': 'importance',
    's': city_name.lower(),
    'a': 'true',
    'format': 'json'})
location_url

'https://locator-service.api.bbci.co.uk/locations?api_key=AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv&stack=aws&locale=en&filter=international&place-types=settlement%2Cairport%2Cdistrict&order=importance&s=pune&a=true&format=json'

In [ ]:
# convert url into json format
result_json = requests.get(location_url).json()
result_json

{'response': {'results': {'results': [{'container': 'India',
     'containerId': 1269750,
     'country': 'IN',
     'id': '1259229',
     'language': 'en',
     'latitude': 18.51957,
     'longitude': 73.85535,
     'name': 'Pune',
     'placeType': 'settlement',
     'timezone': 'Asia/Kolkata'},
    {'container': 'India',
     'containerId': 1269750,
     'country': 'IN',
     'id': '6452628',
     'language': 'en',
     'latitude': 18.58223,
     'longitude': 73.91988,
     'name': 'Pune Airport',
     'placeType': 'airport',
     'timezone': 'Asia/Kolkata'}],
   'totalResults': 2}}}

In [ ]:
# we will get id from the results dictionary
result_json['response']['results']['results'][0]['id']

'1259229'

In [ ]:
# create a function to get the id
def get_id(city):
    city = city.lower()
    location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
    'stack': 'aws',
    'locale': 'en',
    'filter': 'international',
    'place-types': 'settlement,airport,district',
    'order': 'importance',
    's': city,
    'a': 'true',
    'format': 'json'})
    city_url_json = requests.get(location_url).json()
    city_id = city_url_json['response']['results']['results'][0]['id']
    return city_id
city = "New Delhi"
city_id = get_id(city)
print(city_id)

1261481


In [ ]:
# get the url from which we are going to scrape data

url = 'https://www.bbc.com/weather/' + city_id
#print(url)
response = requests.get(url)

Initiate Beautiful Soup

In [ ]:
soup = bs(response.content, 'html.parser')
daily_high_values = soup.find_all(class_ = 'wr-day-temperature__high-value')
daily_low_values =  soup.find_all(class_ = 'wr-day-temperature__low-value')

# get gaily high temperature values
daily_high = []
for row in daily_high_values:
    high = row.span.text
    daily_high.append(high)
print(f'Daily High temperature is: {daily_high}')

# get daily low temperature values
daily_low = []
for row in daily_low_values:
    low = row.span.text
    daily_low.append(low)
print(f'Daily Low temperature is: {daily_low}')

Daily High temperature is: ['16° 60°', '18° 64°', '18° 64°', '19° 65°', '20° 68°', '19° 66°', '19° 67°', '19° 66°', '20° 68°', '21° 69°', '21° 70°', '20° 69°', '20° 67°', '20° 68°']
Daily Low temperature is: ['5° 42°', '7° 44°', '7° 45°', '7° 45°', '7° 45°', '6° 43°', '7° 44°', '7° 44°', '8° 46°', '9° 49°', '9° 48°', '8° 46°', '7° 45°', '8° 46°']


In [ ]:
daily_high[0].strip().split()[0]

'16°'

In [ ]:
daily_high = [i.strip().split()[0] for i in daily_high]
print(f'Daily High temperature(°C) is: {daily_high}')

daily_low = [i.strip().split()[0] for i in daily_low]
print(f'Daily Low temperature(°C) is: {daily_low}')

Daily High temperature(°C) is: ['16°', '18°', '18°', '19°', '20°', '19°', '19°', '19°', '20°', '21°', '21°', '20°', '20°', '20°']
Daily Low temperature(°C) is: ['5°', '7°', '7°', '7°', '7°', '6°', '7°', '7°', '8°', '9°', '9°', '8°', '7°', '8°']


In [ ]:
# find summary
daily_summary = soup.find('div', attrs={'class' : 'wr-day-summary'})
daily_summary.text

'Sunny and light windsSunny and light windsSunny and light windsSunny and light windsSunny and light windsSunny and light windsSunny intervals and light windsSunny intervals and light windsSunny intervals and light windsSunny intervals and light windsSunny intervals and light windsSunny intervals and light windsSunny and light windsSunny and light winds'

In [ ]:
# print(type(summary))
temp = ""
# for i in range(summary):
#     if 

# Scrapping IMDB Data<br>
We will represent 250 list of movies in a tabular form using python.<br> We will use BeautifulSoup library to scrape data from web.


In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
req = requests.get("https://www.imdb.com/chart/top/")

# convert to a beautiful soup object
soup = bs(req.content)

# Print out HTML

contents = soup.prettify() # adds indentation to the data
print(contents)

In [ ]:
movie_title = []
movie_year = []
movie_rating = []

Extract HTML Tag contents

In [ ]:
# The movie name and year are nested under the parent class "chart full_width"
imdb_table = soup.find(class_ = "chart full-width")

In [ ]:
# tag for name of movie
movie_title_column = imdb_table.find_all(class_ = "titleColumn")
#print(movie_title_column)

In [ ]:
# tag for IMDB rating of movie
movie_rating_columns = imdb_table.find_all(class_ = "ratingColumn imdbRating")

In [ ]:
# Extract the titles of movies from movie_title_column
for row in movie_title_column:
    title = row.a.text # extract the text from tag "a"
    movie_title.append(title)
#movie_title = list(movie_title)
len(movie_title)

250

In [ ]:
movie_title # Movie

['The Shawshank Redemption',
 'The Godfather',
 'The Godfather: Part II',
 'The Dark Knight',
 '12 Angry Men',
 "Schindler's List",
 'The Lord of the Rings: The Return of the King',
 'Pulp Fiction',
 'The Good, the Bad and the Ugly',
 'The Lord of the Rings: The Fellowship of the Ring',
 'Fight Club',
 'Forrest Gump',
 'Inception',
 'The Lord of the Rings: The Two Towers',
 'Star Wars: Episode V - The Empire Strikes Back',
 'The Matrix',
 'Goodfellas',
 "One Flew Over the Cuckoo's Nest",
 'Spider-Man: No Way Home',
 'Seven Samurai',
 'Se7en',
 'The Silence of the Lambs',
 'City of God',
 "It's a Wonderful Life",
 'Life Is Beautiful',
 'Saving Private Ryan',
 'Star Wars: Episode IV - A New Hope',
 'Interstellar',
 'Spirited Away',
 'The Green Mile',
 'Parasite',
 'Léon: The Professional',
 'Hara-Kiri',
 'The Pianist',
 'Terminator 2: Judgment Day',
 'Back to the Future',
 'The Usual Suspects',
 'Psycho',
 'The Lion King',
 'Modern Times',
 'Grave of the Fireflies',
 'American History X'

In [ ]:
# Extract the year of the movir release
for row in movie_title_column:
    year = row.span.text # tag for movie year
    movie_year.append(year)

In [ ]:
movie_year # Year

In [ ]:
# Extract the imbd rating
for row in movie_rating_columns:
    rating = row.strong.text # "strong" is tag for rating
    movie_rating.append(rating)

In [ ]:
movie_rating # IMDB

Merge all in a single data frame

In [ ]:
movie_df = pd.DataFrame({"Movie Name" : movie_title, "Year of Release" : movie_year, "IMDB Rating" : movie_rating})
movie_df.head(10)

,Movie Name,Year of Release,IMDB Rating
0,The Shawshank Redemption,(1994),9.2
1,The Godfather,(1972),9.1
2,The Godfather: Part II,(1974),9.0
3,The Dark Knight,(2008),9.0
4,12 Angry Men,(1957),8.9
5,Schindler's List,(1993),8.9
6,The Lord of the Rings: The Return of the King,(2003),8.9
7,Pulp Fiction,(1994),8.8
8,"The Good, the Bad and the Ugly",(1966),8.8
9,The Lord of the Rings: The Fellowship of the Ring,(2001),8.8


# Scrapping Wikipedia

In [ ]:
# import wikipedia library
!pip install wikipedia
import wikipedia as wk

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=cc02dcd849f4ab14cb5b6fbab33553b2ccd56b5326b37913a2c55b3d5ae89baa
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [ ]:
# search any keyword
print(wk.search("COEP"))

['College of Engineering, Pune', 'Sindhutai Sapkal', 'Anil Sahasrabudhe', 'Maharashtra', 'List of Indian engineering colleges before Independence', 'Prohibition in the United States', 'Community organizing', 'Swayam', 'List of Maharashtra Government Engineering Colleges', 'Harish S. Mehta']


In [ ]:
print(wk.search("COEP", results = 3))

['College of Engineering, Pune', 'Sindhutai Sapkal', 'Anil Sahasrabudhe']


In [ ]:
print(wk.summary("College Of Engineering Pune"))

College of Engineering Pune (COEP), is an autonomous institute of the Government of Maharashtra, situated in Pune, Maharashtra, India. Established in 1854 it is the 3rd oldest engineering college in India, after IIT Roorkee (1847) and College of Engineering, Guindy, Chennai (1859). The students and alumni of College of Engineering, Pune are colloquially referred to as COEPians. It was the first technical institution in India to offer mechanical engineering.


In [ ]:
print(wk.summary("College Of Engineering Pune", sentences = 2))

College of Engineering Pune (COEP), is an autonomous institute of the Government of Maharashtra, situated in Pune, Maharashtra, India. Established in 1854 it is the 3rd oldest engineering college in India, after IIT Roorkee (1847) and College of Engineering, Guindy, Chennai (1859).


In [ ]:
# to get complete page
coep_page = wk.page("College Of Engineering Pune")
print(coep_page.content)

College of Engineering Pune (COEP), is an autonomous institute of the Government of Maharashtra, situated in Pune, Maharashtra, India. Established in 1854 it is the 3rd oldest engineering college in India, after IIT Roorkee (1847) and College of Engineering, Guindy, Chennai (1859). The students and alumni of College of Engineering, Pune are colloquially referred to as COEPians. It was the first technical institution in India to offer mechanical engineering.


== History ==

The college was one of the first few colleges established by the British to serve the Indian subcontinent. The foundation stone of the new college was laid by the Governor of Bombay, Sir Bartle Frere, who was to become the Vice-Chancellor of the University, on 5 August 1865. In June of the following year, Theodore Cooke M.A., was appointed the Principal. He went on to hold the office for next 28 years.

Initially, it was started on 16 December 1854, to train public works department (PWD) officials and was housed in 

In [ ]:
# get url of page
print(coep_page.url)

https://en.wikipedia.org/wiki/College_of_Engineering,_Pune


In [ ]:
# get title of page
print(coep_page.title)

College of Engineering, Pune


In [ ]:
# get images of keyword
print(coep_page.images)

['https://upload.wikimedia.org/wikipedia/commons/2/27/Bust_of_Sir_Visvesvarayya_facing_the_College%27s_Main_building.jpg', 'https://upload.wikimedia.org/wikipedia/commons/a/a7/COEP_Date.JPG', 'https://upload.wikimedia.org/wikipedia/commons/4/4e/COEP_Main_building.JPG', 'https://upload.wikimedia.org/wikipedia/commons/c/ca/College_of_Engineering%2C_Pune_Auditorium_%28Stage_View%29.jpg', 'https://upload.wikimedia.org/wikipedia/commons/c/ca/People_at_83rd_Regatta.jpg', 'https://upload.wikimedia.org/wikipedia/commons/6/63/Pune_ShaniwarWada_DelhiGate.jpg', 'https://upload.wikimedia.org/wikipedia/commons/0/04/The_Mind-Spark_Champion%27s_Trophy.jpg', 'https://upload.wikimedia.org/wikipedia/en/f/f0/College_of_Engineering%2C_Pune_logo.jpg', 'https://upload.wikimedia.org/wikipedia/en/4/41/Flag_of_India.svg', 'https://upload.wikimedia.org/wikipedia/en/8/8a/OOjs_UI_icon_edit-ltr-progressive.svg']


In [ ]:
# the tables on the page will be stored as list
# and can be converttd by using pandas

coep_html = wk.page("College Of Engineering Pune").html().encode("UTF-8")
coep_html
import pandas as pd
coep_df = pd.read_html(coep_html)[4]
coep_df

,0,1
0,Palaces and forts,Shaniwar Wada Vishrambaug Wada Shinde Chhatri ...
1,Temples,Puneshwar Narayaneshwar Pataleshwar Kasba Gana...
2,Museums,Raja Dinkar Kelkar Museum Mahatma Phule Museum...
3,Parks,Bund Garden Rajiv Gandhi Zoological Park Shuny...
4,Others,Bal Gandharva Ranga Mandir Tilak Smarak Ranga ...


# Basic Pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
excel_file = '/content/drive/MyDrive/Colab Notebooks/Diploma/Data Sets/IFCB2009_85.xlsx'
excel_file_df = pd.read_excel(excel_file)
excel_file_df.head()

In [ ]:
excel_file_df.shape

In [ ]:
excel_file_df['CITY 1'].value_counts()

In [ ]:
locator = Nominatim(user_agent="http")
location = locator.geocode('CHENNAI')
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

# Scraping PDFs

In [ ]:
import os
!pip install tabula-py
import tabula
import urllib.request
from urllib.parse import urljoin
url = 'https://www.premierleague.com/publications'

# Specify the folder for the files to be downloaded
folder_loc = r'/content/drive/MyDrive/Colab Notebooks/Diploma/Tools in Data Science/Premier_League'

# if not os.path.exists(folder_loc):
#     os.mkdir(folder_loc)

Actual Code

In [ ]:
response = requests.get(url)
soup = bs(response.text, 'html.parser')

# extract all tags that end in .pdf
for link in soup.select('a[href$=".pdf"]'):
    # Name of the file to be downloaded
    filename = os.path.join(folder_loc, link['href'].split('/')[-1])
    print(filename)
    with open(filename, 'wb') as f:
        f.write(requests.get(urljoin(url, link['href'])).content)

Read table from file and export it to PDF

In [ ]:
import tabula
df = tabula.read_pdf(folder_loc + '/This-is-PL-Interactive-Combined.pdf', pages='18')
df